# Imports

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
from shapely.ops import unary_union
import matplotlib.pyplot as plt
from shapely import wkt

# Constants

In [2]:
CRS = 4326
COLORS = {
    'green': '#83bca9',
    'darkgreen': '#3e5641'
}

## Load in USA Parks data

In [5]:
usa_parks = gpd.read_file("../data/USA_Parks/v10/park_dtl.gdb/")
usa_parks = usa_parks.to_crs(epsg=CRS) # convert to const CRS value (4326)

## Load US State Boundaries

In [7]:
us_states = pd.read_csv("../data/United_States_Boundary_Files.csv")
us_states['geometry'] = us_states['the_geom'].apply(wkt.loads) # create geometry column with safety
us_states_gdf = gpd.GeoDataFrame(us_states, crs = CRS) # convert to GeoPd DF with const CRS value (4326)

### Find parks only in or bordering TN

In [1]:
tn_parks = usa_parks.sjoin( # sjoin to find the parks that 'intersect' or are in and on the border of TN
                     us_states_gdf.loc[us_states_gdf['STUSPS'] == 'TN'],
                     predicate = 'intersects')

tn_parks['name'] = tn_parks['NAME_left'] # rename name column
tn_parks = tn_parks[['name', 'FEATTYPE', 'SQMI', 'geometry']] # keep only neccessary columns

NameError: name 'usa_parks' is not defined

# Create a map of the TN parks

In [26]:
# tn_map = folium.Map(tiles = "Cartodb Positron",
#                      location = (36, -86.1),
#                      zoom_start = 7,
#                      prefer_canvas=True)

In [28]:
# add feature group
# tn_layer = folium.FeatureGroup(name = 'Tennessee Parks', show = True)

# for i, park in tn_parks.iterrows():    # iterate throug rows of TN parks GeoDF
#     textbox = folium.Popup(
#             park['name']
#     )
    
#     folium.GeoJson(
#             park['geometry'],
#             popup = textbox,
#             style_function=lambda x: {"fillColor": c['green'],
#                                                                  'color':COLORS['darkgreen'],
#                                                                 'weight': 0.5}).add_to(tn_layer)

# tn_layer.add_to(tn_map)
# folium.LayerControl().add_to(tn_map)

# Load in Places Data

In [13]:
places_df = pd.read_csv('../data/PLACES__Local_Data_for_Better_Health__Place_Data_2023_release_20240504.csv')
# places_tact = pd.read_csv('../data/PLACES__Local_Data_for_Better_Health__Census_Tract_Data_2023_release_20240504.csv')
# data_dict = pd.read_csv('../data/PLACES_and_500_Cities__Data_Dictionary_20240504.csv')

#### Create places geopandas DF 

In [14]:
places_df['geometry'] = places_df['Geolocation'].apply(wkt.loads) # create geometry column with safety
places_gdf = gpd.GeoDataFrame(places_df, crs = CRS) # convert to GeoPd DF with const CRS value (4326)

### Find Places data only in TN and filter out unneccessary columns

In [15]:
tn_places = places_gdf.loc[places_gdf['StateAbbr'] == 'TN'][['Year', 'StateAbbr',
                                                             'Category', 'Measure',
                                                             'Data_Value', 'TotalPopulation',
                                                            'geometry', 'LocationID', 'MeasureId']]

####  Find the min distance between each Health data point and the nearest park. Save as CSV

In [16]:
# tn_health_loc = tn_places[['LocationID', 'geometry']] # create smaller Df to iterate through
# tn_parks_loc = tn_parks[['name', 'geometry']] # create smaller Df to iterate through

In [31]:
# min_dist = [] # empty list to hold the min distances to create a new column out of
# for i, r_health in tn_health_loc.iterrows(): # iterate through each row of the TN health data points
#     distances = [] # create empty list to hold the distnace from the 
#                    # health point and each state park boundary
#     for idx, r_park in tn_parks_loc.iterrows(): # iterate though each row of the TN parks data
#         distances.append(r_health['geometry'].distance(r_park['geometry'])) # add the distance from 
#                                                                             # the health point top
#                                                                             # each park to the list
#     min_dist.append(np.min(distances)) # find the distance from the health point to the nearest park

# tn_health_loc['distance'] = min_dist # create new column in the TN health points DF
# tn_health_loc.to_csv('../data/TN_health_data_with_distances_to_parks.csv') # save as csv for future recals

#### Measures to Focus on

In [152]:
# list of measures to focus on
measures = ['Cancer (excluding skin cancer) among adults aged >=18 years',
           'Cholesterol screening among adults aged >=18 years',
           'Coronary heart disease among adults aged >=18 years',
           'Current asthma among adults aged >=18 years',
           'Current lack of health insurance among adults aged 18-64 years',
           'Depression among adults aged >=18 years',
           'Diagnosed diabetes among adults aged >=18 years',
           'Fair or poor self-rated health status among adults aged >=18 years',
           'High blood pressure among adults aged >=18 years',
           'Obesity among adults aged >=18 years',
           'Stroke among adults aged >=18 years']

#### Filter the DF by measures

In [ ]:
places_filtered = places_df.loc[places_df['Measure'].isin(measures)]